## Introduction to Amazon Simple Storage Service (S3)

#### INFORMACIÓN GENERAL
Amazon Simple Storage Service (Amazon S3) es un servicio de almacenamiento de objetos que ofrece escalabilidad, disponibilidad de datos, seguridad y rendimiento líderes en el sector. Es decir que los clientes de todos los tamaños y sectores pueden almacenar y proteger cualquier cantidad de datos para una serie de casos de uso, como sitios web, aplicaciones móviles, procesos de copia de seguridad y restauración, archivos, aplicaciones para empresas, dispositivos del Internet de las cosas (IoT) y análisis de big data. Amazon S3 proporciona características de administración fáciles de utilizar que le permiten organizar los datos y configurar controles de accesos precisos para satisfacer sus requisitos empresariales, organizativos y de conformidad específicos. Amazon S3 está diseñado para ofrecer una durabilidad del 99,999999999 % (11 nueves) y almacena los datos de millones de aplicaciones para empresas de todo el mundo.

#### SITUACIÓN DE LABORATORIO
Usted trabaja en una empresa que utiliza Amazon S3 para el almacenamiento de datos. Una aplicación que se encuentra en una instancia EC2 tiene que enviar a diario los datos de informes a un bucket de S3. Se ocupa de crear un bucket de S3 a fin de que su empresa lo utilice para almacenar estos datos de informes. Para una implementación exitosa, tiene que garantizar que la instancia EC2 tenga suficientes privilegios como para poder cargar y recuperar los datos del bucket de S3. Por razones de seguridad, solo la instancia EC2 puede escribir datos en el bucket de S3. Los archivos del bucket de S3 también requieren protección contra la eliminación accidental. Este laboratorio sigue al curso digital Getting Started with Amazon S3.

In [1]:
import boto3
import json
import random

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable = vpc.create_route_table()
routetable.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT"}])

[ec2.Tag(resource_id='rtb-01952e9e45da52423', key='Name', value='AWSCookbookVPC-RT')]

In [4]:
# Cree una subrede:
subnet = ec2.create_subnet(
    CidrBlock='10.10.1.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN"}])

[ec2.Tag(resource_id='subnet-08d4c9c2037eeaa42', key='Name', value='AWSCookbook-SN')]

In [5]:
# Asociar la tabla de rutas con la subred:
routetable.associate_with_subnet(SubnetId = subnet.id)

ec2.RouteTableAssociation(id='rtbassoc-0da6e1ef60d8425e4')

In [7]:
# Crear un internet gateway:
internetgateway = ec2.create_internet_gateway()
internetgateway.wait_until_available()  # Esperar a que el gateway esté disponible
internetgateway.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-IGW"}])

[ec2.Tag(resource_id='igw-0cfb762647424783e', key='Name', value='AWSCookbook-IGW')]

In [8]:
# Asociar el gateway de internet con la VPC:
internetgateway.attach_to_vpc(VpcId=vpc.id)

{'ResponseMetadata': {'RequestId': 'e9ddd90c-0afa-4d8f-8ee8-46bcb549ceef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e9ddd90c-0afa-4d8f-8ee8-46bcb549ceef',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Wed, 12 Oct 2022 11:34:37 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [9]:
# Cree una ruta de Internet:
routetable.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=internetgateway.id
)

ec2.Route(route_table_id='rtb-01952e9e45da52423', destination_cidr_block='0.0.0.0/0')

#### Tarea 1: Crear un bucket

 Los nombres de bucket deben tener entre 3 y 63 caracteres, y contener solo letras minúsculas, números o guiones. El nombre del bucket debe ser único en forma global en todo Amazon S3, independientemente de la cuenta o la región, y no se puede cambiar una vez creado el bucket.

In [18]:
# Create a bucket
bucket_name = 'reportbucket-{}'.format(random.randint(1000, 100000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

Bucket name: reportbucket-26053


Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) están desactivadas por defecto. Tanto las ACL como las políticas de bucket se utilizan más adelante en el laboratorio, por lo que todas permanecen sin seleccionar en esta tarea. En un entorno de producción, se recomienda utilizar la configuración con la menor cantidad de permisos posible.

In [13]:
s3_client.put_public_access_block(
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': False,
        'IgnorePublicAcls': False,
        'BlockPublicPolicy': False,
        'RestrictPublicBuckets': False
    }
)

{'ResponseMetadata': {'RequestId': 'GDXPV6WX8VYCZHVE',
  'HostId': 'GQGCeuurCLkc4EFbishuezzOJBO0pNGUro8h3j20bM8iXYU1okS1wTyQESTacP2rXEYLxP3CNOs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GQGCeuurCLkc4EFbishuezzOJBO0pNGUro8h3j20bM8iXYU1okS1wTyQESTacP2rXEYLxP3CNOs=',
   'x-amz-request-id': 'GDXPV6WX8VYCZHVE',
   'date': 'Wed, 12 Oct 2022 11:40:03 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

#### Tarea 2: Cargar un objeto en el bucket

In [20]:
# Upload the file to the bucket
file_name = 'new-report.png'
s3.Object(bucket_name, file_name).put(Body=open(file_name, 'rb'))

{'ResponseMetadata': {'RequestId': 'WDWV2B4WE6X01WV0',
  'HostId': 'hoNEZrBVSUInJco+idNLYBVXWSbxv5Q8Iuf/GoppDtpiUxhTGsxlaEPCty+xp6QKgmvbr8K21aM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hoNEZrBVSUInJco+idNLYBVXWSbxv5Q8Iuf/GoppDtpiUxhTGsxlaEPCty+xp6QKgmvbr8K21aM=',
   'x-amz-request-id': 'WDWV2B4WE6X01WV0',
   'date': 'Wed, 12 Oct 2022 11:52:19 GMT',
   'etag': '"75acf5a0dd2f6bdd67c36fa2748a1a19"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"75acf5a0dd2f6bdd67c36fa2748a1a19"'}

#### Tarea 3: Hacer que un objeto sea público

In [21]:
# Get url for the file in the bucket
url = "https://{}.s3.amazonaws.com/{}".format(bucket_name, file_name)
print("URL: {}".format(url))

URL: https://reportbucket-26053.s3.amazonaws.com/new-report.png


* Abra una nueva pestaña del navegador, pegue el enlace de la URL del objeto en el campo de dirección y, a continuación, presione Intro.

* A continuación, verá el error Access Denied (Acceso denegado). Esto se debe a que los objetos de Amazon S3 son privados de forma predeterminada.

* Este error se muestra debido a que este bucket está configurado para no permitir el acceso público. La configuración del bucket anula los permisos aplicados a objetos individuales. Si quiere que el objeto sea visible para el público general, tiene que desactivar el bloqueo de acceso público (BPA).

In [22]:
# Make the file public:
s3.Object(bucket_name, file_name).Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'Q2HKFWE6Q0YPPANY',
  'HostId': '2yed8K681XXLqQpaj33NbzafLvyJy4CuTw1J15PeruBG+207LU9q+5qHzyYD3CAxwvJxyoKe0dY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2yed8K681XXLqQpaj33NbzafLvyJy4CuTw1J15PeruBG+207LU9q+5qHzyYD3CAxwvJxyoKe0dY=',
   'x-amz-request-id': 'Q2HKFWE6Q0YPPANY',
   'date': 'Wed, 12 Oct 2022 11:52:49 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

Regrese a la otra pestaña del navegador en la que se muestra Access Denied (Acceso denegado) para new-report.png y actualice la página, new-report.png ahora se muestra de forma correcta porque es accesible públicamente.

En este ejemplo, concedió acceso de lectura únicamente para un objeto específico. Si quiere conceder acceso a todo el bucket, tiene que utilizar una política de bucket, lo cual se cubre más adelante en este laboratorio.

#### Probar la conectividad con el bucket S3

In [24]:
iam = boto3.client('iam')

In [25]:
# Create a security group:
secgroup = ec2.create_security_group(
    GroupName='AWSCookbook-SG-HHTPS',
    Description='Allow HTTP and HTTPS',
    VpcId=vpc.id
)

In [26]:
# Add rules to the security group:
secgroup.authorize_ingress(
    CidrIp='0.0.0.0/0',
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80
)

secgroup.authorize_ingress(
    CidrIp='0.0.0.0/0',
    IpProtocol='tcp',
    FromPort=443,
    ToPort=443
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-05680d82d16c5c2cc',
   'GroupId': 'sg-02e209afd5638aa64',
   'GroupOwnerId': '782265949238',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': 'ab728051-0ddc-409a-abfb-3c1ed977c6d5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ab728051-0ddc-409a-abfb-3c1ed977c6d5',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '721',
   'date': 'Wed, 12 Oct 2022 11:54:43 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [27]:
# Create a role
role_name = 'SSM-Role-EC2-Public'
policy_document = {
"Version": "2012-10-17",
"Statement": [
        {   
            "Effect": "Allow",
            "Principal": {
            "Service": "ec2.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

In [28]:
instance_profile_name = 'Profile-SSM-Role-Public'
ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

# Attach the policy to the role
response = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

# Create an instance profile
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)
# Add the role that you created to the instance profile:
response = iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

In [29]:
# Create a instance EC2 with IP public and SSM and connect SSM:
instance = ec2.create_instances(
    ImageId='ami-0c2b8ca1dad447f8a',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    SecurityGroupIds=[secgroup.id],
    IamInstanceProfile={
            'Name': instance_profile_name
        },
    SubnetId=subnet.id,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'AWSCookbook-EC2'
                },
            ]
        },
    ]
)

In [30]:
# Wait for the instance to enter the running state
instance[0].wait_until_running()

In [31]:
# Create a IP elastic:
ec2_client = boto3.client('ec2')
response = ec2_client.allocate_address(
    Domain='vpc'
)

In [32]:
# Associate IP elastic with instance EC2:
response = ec2_client.associate_address(
    InstanceId=instance[0].id,
    AllocationId=response['AllocationId']
)

In [37]:
# Reboot instance EC2:
response = instance[0].reboot()

Agregar una politica al rol con el fin de permitir que la instancia EC2 enumere los objetos y los buckets de S3.

In [41]:
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
)

{'ResponseMetadata': {'RequestId': 'ecd0f1be-bd5c-438d-8ca7-7d3b6621c6fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ecd0f1be-bd5c-438d-8ca7-7d3b6621c6fb',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 12 Oct 2022 12:04:53 GMT'},
  'RetryAttempts': 0}}

In [36]:
print("Connecting to the last instance created with SSM...")
print("   aws ssm start-session --target " + instance[0].id)

Connecting to the last instance created with SSM...
   aws ssm start-session --target i-03171797c93592a0f


En la sesión de la instancia, ingrese el siguiente comando para cambiar al directorio principal (/home/ssm-user/):

In [ ]:
cd ~

Ingrese el siguiente comando para comprobar que se encuentra en el directorio principal:

In [ ]:
pwd

Este debe ser el resultado: `/home/ssm-user`

Ingrese el siguiente comando para enumerar todos los buckets de S3.

In [ ]:
aws s3 ls

In [38]:
# Ingrese el siguiente comando para enumerar todos los objetos de reportbucket
print("aws s3 ls s3://{}/".format(bucket_name))

aws s3 ls s3://reportbucket-26053/


Escriba lo siguiente para ver si puede copiar un archivo en el bucket de S3

In [ ]:
wget https://stats.govt.nz/assets/Uploads/Business-operations-survey/Business-operations-survey-2021/Download-data/bos2021ModC.csv

Y a continuación:

In [56]:
file_name = 'bos2021ModC.csv'
print("aws s3 cp {} s3://{}".format(file_name, bucket_name))

aws s3 cp bos2021ModC.csv s3://reportbucket-26053


El resultado indica un error upload failed (error al cargar). Esto se debe a que tenemos derechos de solo lectura para el bucket y no tenemos los permisos para realizar la operación PutObject.

#### Crear una política de bucket

Una política de bucket es un conjunto de permisos asociados a un bucket de S3. Se utiliza para controlar el acceso a un bucket completo o a directorios específicos dentro de un bucket.

Este es el rol que utiliza la instancia EC2 para conectarse a S3:


In [43]:
print(role_name)

SSM-Role-EC2-Public


In [53]:
# Obtener el ARN del rol:
response = iam.get_role(
    RoleName=role_name
)
role_arn = response['Role']['Arn']
print(role_arn)

arn:aws:iam::782265949238:role/SSM-Role-EC2-Public


Los nombres de recursos de Amazon (ARN) identifican de forma exclusiva los recursos de AWS en todo AWS. Cada sección del ARN está separada por “:” y representa una parte específica de la ruta al recurso especificado. Las secciones pueden variar levemente de acuerdo con el servicio al que se hace referencia, pero en general siguen este formato:

* `arn:partition:service:region:account-id:resource`

Amazon S3 no requiere parámetros de region (región) o de account-id en los ARN, por lo que esas secciones se dejan en blanco. Sin embargo, se utilizan “:” para separar las secciones, por lo que se ve similar a `arn:aws:s3:::reportbucket987987`.

In [51]:
policy_document_s3 = {
    "Version": "2012-10-17",
    "Id": "Policy1604361694227",
    "Statement": [
        {
            "Sid": "Stmt1604361692117",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject"
            ],
            "Resource": "arn:aws:s3:::{}/*".format(bucket_name)
        }
    ]
}

* La acción de obtener GetObject concede permiso para obtener objetos de Amazon S3. Consulte la sección Recursos adicionales al final del laboratorio y acceda a enlaces con más información sobre las acciones disponibles para utilizar en las políticas de Amazon S3.

* Un nombre de recurso de Amazon (ARN) es una forma estándar de referirse a los recursos en AWS. En este caso, el ARN se refiere a su bucket de S3. Cuando se agrega /* al final del nombre del bucket, se permite que la política se aplique a todos los objetos dentro del bucket.

In [52]:
# Attach the policy to the role:
response = iam.put_role_policy(
    RoleName=role_name,
    PolicyName='Policy-SSM-instance-S3',
    PolicyDocument=json.dumps(policy_document_s3)
)


Escriba lo siguiente para tratar de copiar el archivo bos2021ModC.csv en el bucket de S3.

In [54]:
print("aws s3 cp {} s3://{}".format(file_name, bucket_name))

aws s3 cp bos2021ModC.csv s3://reportbucket-26053


Ha cargado correctamente (PutObject) un archivo de la instancia EC2 en su bucket de S3.

Ahora escriba el siguiente comando para recuperar (GetObject) un archivo de S3 a la instancia EC2.

In [57]:
file_name_report = 'new-report.png'
print("aws s3 cp s3://{b}/{f} {f}".format(b = bucket_name, f = file_name_report))

aws s3 cp s3://reportbucket-26053/new-report.png new-report.png


Regrese a la pestaña del navegador y pegue la siguiente dirección:

In [59]:
url = "https://{}.s3.amazonaws.com/{}".format(bucket_name, file_name)
print("URL: {}".format(url))

URL: https://reportbucket-26053.s3.amazonaws.com/bos2021ModC.csv


La página sigue mostrando un mensaje de error porque la política de bucket solo otorgó los derechos a la entidad principal que es el rol asociado con la instancia.

A continuación creará una polítca para el bucket que permite acceder a **TODOS** los archivos de forma pública.

In [60]:
# Attach the policy to S3
response = s3.BucketPolicy(bucket_name).put(
    Policy=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "PublicReadGetObject",
                "Effect": "Allow",
                "Principal": "*",
                "Action": "s3:GetObject",
                "Resource": "arn:aws:s3:::{}/*".format(bucket_name)
            }
        ]
    })
)


Pruebe acceder nuevamente a la dirección:

In [61]:
url = "https://{}.s3.amazonaws.com/{}".format(bucket_name, file_name)
print("URL: {}".format(url))

URL: https://reportbucket-26053.s3.amazonaws.com/bos2021ModC.csv


#### Explorar el control de versiones

El control de versiones es una forma de conservar diversas variantes de un objeto en el mismo bucket. Puede utilizar el control de versiones para conservar, recuperar y restablecer todas las versiones de todos los objetos almacenados en su bucket de Amazon S3. Con el control de versiones, puede recuperarse fácilmente de acciones no deseadas del usuario y de errores de la aplicación.

*Por razones de auditoría y conformidad, tiene que habilitar el control de versiones en reportbucket. El control de versiones debe proteger los informes de reportbucket contra la eliminación accidental. Como usuario, desea validar qué tan cierta es esta funcionalidad. En esta tarea, habilitará el control de versiones y probará la característica al cargar una versión modificada del archivo bos2021ModC.csv de la tarea anterior.*

In [64]:
# Activar la auditoría de S3:
response = s3.BucketVersioning(bucket_name).enable()
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Bucket versioning enabled")

Bucket versioning enabled


* El control de versiones está habilitado para un bucket completo y todos los objetos dentro del bucket. No se puede habilitar para los objetos individuales.

* También hay consideraciones de costo al habilitar el control de versiones. Consulte las [preguntas frecuentes](https://aws.amazon.com/es/s3/faqs/) para más información.

Copie los siguientes comandos en la terminal:

In [ ]:
cat > sample-file.txt << EOF

This sample text file is used to illustrate the use of versioning in an Amazon S3 bucket.

Make it a great day!

EOF


In [66]:
print("aws s3 cp sample-file.txt s3://{}/sample-file.txt".format(bucket_name))

aws s3 cp sample-file.txt s3://reportbucket-26053/sample-file.txt


In [ ]:
cat > sample-file.txt << EOF

This sample text file is used to illustrate the use of versioning in an Amazon S3 bucket.

This file has been modified.

This is version 2 of the file.

Have a lovely day!

EOF


In [67]:
print("aws s3 cp sample-file.txt s3://{}/sample-file.txt".format(bucket_name))

aws s3 cp sample-file.txt s3://reportbucket-26053/sample-file.txt


In [73]:
# List versions of the file:
response = s3_client.list_object_versions(
    Bucket=bucket_name,
    Prefix='sample-file.txt'
)

for version in response['Versions']:
    print("Version ID: {}".format(version['VersionId']))
    print("Last modified: {}".format(version['LastModified']))
    print("Is latest: {}".format(version['IsLatest']))
    print("")

Version ID: tHxR7_KH0cT.gvxVZN8vRm7W9UrV0oVW
Last modified: 2022-10-12 12:58:53+00:00
Is latest: True

Version ID: iikIbrac2r7RdTQtdppp0SUtGw6cfbr0
Last modified: 2022-10-12 12:57:04+00:00
Is latest: False



Amazon S3 siempre devuelve la versión más reciente de un objeto si no se especifica una versión

In [75]:
# Download a file:
response = s3_client.get_object(
    Bucket=bucket_name,
    Key='sample-file.txt',
)

# Save the file:
with open('sample-file.txt', 'wb') as f:
    f.write(response['Body'].read())

Para descargar la primer versión:

In [76]:
# Download a file:
response = s3_client.get_object(
    Bucket=bucket_name,
    Key='sample-file.txt',
    VersionId='iikIbrac2r7RdTQtdppp0SUtGw6cfbr0'
)

# Save the file:
with open('sample-file.txt', 'wb') as f:
    f.write(response['Body'].read())

Para hacer la versión primera la principal:

In [77]:
response = s3_client.copy_object(
    Bucket=bucket_name,
    CopySource={
        'Bucket': bucket_name,
        'Key': 'sample-file.txt',
        'VersionId': 'iikIbrac2r7RdTQtdppp0SUtGw6cfbr0'
    },
    Key='sample-file.txt'
)

In [79]:
# Download a file:
response = s3_client.get_object(
    Bucket=bucket_name,
    Key='sample-file.txt',
)

# Save the file:
with open('sample-file.txt', 'wb') as f:
    f.write(response['Body'].read())